In [1]:
from behavior_and_time_alignment import beh_and_time_alignment
from session_preprocessing import ephys_opto_preprocessing, session_crosscorr, ephys_opto_crosscorr
from opto_tagging import opto_plotting_session, opto_plotting_unit, opto_tagged_spike_stability
from opto_waveforms_preprocessing import opto_wf_preprocessing
from drift_analysis import plot_session_opto_drift
import matplotlib.pyplot as plt
from utils.beh_functions import session_dirs
import numpy as np
import pandas as pd
import os
from utils.plot_utils import combine_pdf_big
import spikeinterface as si
from joblib import Parallel, delayed



In [2]:
session_assets = pd.read_csv('/root/capsule/code/data_management/session_assets.csv')
session_list = session_assets['session_id']
ani_list = []

In [3]:
# combine opto pdfs
data_type = 'curated'
def process(session, data_type):
    pdf_dir = session_dirs(session)[f'opto_dir_fig_{data_type}']
    output_pdf = os.path.join(session_dirs(session)[f'opto_dir_{data_type}'], f'{session}_opto_tagging_png.pdf')
    if os.path.exists(pdf_dir) and not os.path.exists(output_pdf):
        print(session)
        combine_pdf_big(pdf_dir, output_pdf)
# Parallel(n_jobs=4)(delayed(process)(session, data_type) for session in session_list)
for session in session_list:
    try:
        process(session, data_type)
        plt.close('all')
    except:
        print(f'Failed to process {session}')

behavior_716325_2024-05-29_10-33-32
No PDFs found in the directory!
behavior_716325_2024-05-30_11-33-46
No PDFs found in the directory!
behavior_716325_2024-05-31_10-31-14
No PDFs found in the directory!
behavior_716325_2024-06-01_09-25-37
No PDFs found in the directory!
behavior_717121_2024-06-11_10-23-31
No PDFs found in the directory!
behavior_717121_2024-06-13_12-21-20
No PDFs found in the directory!
behavior_717121_2024-06-14_10-23-49
No PDFs found in the directory!
behavior_717121_2024-06-16_11-45-02
No PDFs found in the directory!
behavior_717259_2024-06-25_10-09-25
No PDFs found in the directory!
behavior_717259_2024-06-26_10-23-07
No PDFs found in the directory!
behavior_717259_2024-06-28_11-17-19
No PDFs found in the directory!
behavior_717263_2024-07-23_11-46-42
No PDFs found in the directory!
behavior_717263_2024-07-24_10-40-05
No PDFs found in the directory!
behavior_717263_2024-07-25_10-02-21
No PDFs found in the directory!
behavior_714116_2024-08-27_11-29-48
No PDFs foun

In [9]:
process('behavior_751004_2024-12-19_11-50-37', data_type) 

behavior_751004_2024-12-19_11-50-37
Combined PDF saved as: /root/capsule/scratch/751004/behavior_751004_2024-12-19_11-50-37/ephys/opto/curated/behavior_751004_2024-12-19_11-50-37_opto_tagging_png.pdf


In [9]:
from joblib import Parallel, delayed
# session_list = ['behavior_717121_2024-06-15_10-00-58',
#                 'behavior_716325_2024-05-31_10-31-14']
data_type = 'curated'
def process(session, data_type='raw'):
    # beh_and_time_alignment(session)
    # plt.close('all')
    # ephys_opto_preprocessing(session, data_type, 'soma')
    # plt.close('all')
    # ephys_opto_crosscorr(session, data_type)
    # plt.close('all')
    # opto_wf_preprocessing(session, data_type, 'soma', load_sorting_analyzer=True)
    
    plt.close('all')
    print(f'{session} start')
    opto_tagging_df = opto_plotting_session(session, data_type, 'soma', plot=True, resp_thresh=0.4, lat_thresh=0.02)
    # save as pickle
    with open(os.path.join(session_dirs(session)[f'opto_dir_{data_type}'], f'{session}_opto_tagging_metrics.pkl'), 'wb') as f:
        opto_tagging_df.to_pickle(f)
    # plt.close('all')                               
    # opto_tagged_spike_stability(session, data_type, 'soma', opto_tagging_df=opto_tagging_df)
    # if '751004' in session:
    #     plot_session_opto_drift(session, data_type, update_csv=False)
    # else:
    #     plot_session_opto_drift(session, data_type, update_csv=True)
    print(f'{session} stop')
        

Parallel(n_jobs=2)(delayed(process)(session, data_type = data_type) for session in session_list[2:])


behavior_751004_2024-12-22_13-09-17 start
behavior_751004_2024-12-21_13-28-28 start


/opt/conda/lib/python3.10/site-packages/spikeinterface/core/base.py:1109: UserWarning: Versions are not the same. This might lead to compatibility errors. Using spikeinterface==0.101.2 is recommended
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/spikeinterface/core/base.py:1109: UserWarning: Versions are not the same. This might lead to compatibility errors. Using spikeinterface==0.101.2 is recommended
  warnings.warn(
/root/capsule/code/beh_ephys_analysis/opto_tagging.py:351: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  unit_qc = unit_qc.apply(pd.to_numeric, errors='ignore')
/root/capsule/code/beh_ephys_analysis/opto_tagging.py:390: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  opto_wf = opto_wf.apply(pd.to_numeric, errors='ignore')


25 out of 110 units pass quality control
{'samples_to_keep': [-30, 60], 'y_neighbors_to_keep': 3, 'orginal_loc': False}


/root/capsule/code/beh_ephys_analysis/opto_tagging.py:351: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  unit_qc = unit_qc.apply(pd.to_numeric, errors='ignore')


65 out of 176 units pass quality control
{'samples_to_keep': [-30, 60], 'y_neighbors_to_keep': 3, 'orginal_loc': False}


/root/capsule/code/beh_ephys_analysis/opto_tagging.py:390: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  opto_wf = opto_wf.apply(pd.to_numeric, errors='ignore')


KeyboardInterrupt: 

In [5]:
session_list

['behavior_751004_2024-12-19_11-50-37',
 'behavior_751004_2024-12-20_13-26-11',
 'behavior_751004_2024-12-21_13-28-28',
 'behavior_751004_2024-12-22_13-09-17',
 'behavior_751004_2024-12-23_14-20-03',
 'behavior_751769_2025-01-16_11-32-05',
 'behavior_751769_2025-01-17_11-37-39',
 'behavior_751769_2025-01-18_10-15-25',
 'behavior_758017_2025-02-04_11-57-38',
 'behavior_758017_2025-02-05_11-42-34',
 'behavior_758017_2025-02-06_11-26-14',
 'behavior_758017_2025-02-07_14-11-08',
 'behavior_751766_2025-02-11_11-53-38',
 'behavior_751766_2025-02-13_11-31-21',
 'behavior_751766_2025-02-14_11-37-11',
 'behavior_751766_2025-02-15_12-08-11']

In [ ]:
session_list

['behavior_717121_2024-06-15_10-00-58', 'behavior_716325_2024-05-31_10-31-14']

In [5]:
process('behavior_717121_2024-06-15_10-00-58', data_type='curated')
# process('behavior_717121_2024-06-15_10-00-58')

/opt/conda/lib/python3.9/site-packages/scipy/stats/_stats_py.py:7985: RuntimeWarning: invalid value encountered in divide
  terms = (f_obs_float - f_exp)**2 / f_exp
/opt/conda/lib/python3.9/site-packages/spikeinterface/core/base.py:1129: UserWarning: Versions are not the same. This might lead to compatibility errors. Using spikeinterface==0.100.7 is recommended
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/spikeinterface/core/waveforms_extractor_backwards_compatibility.py:650: UserWarning: Removing legacy parameter return_scaled from spike_amplitudes extension
  warnings.warn(f"Removing legacy parameter {p} from {old_name} extension")
/opt/conda/lib/python3.9/site-packages/spikeinterface/core/waveforms_extractor_backwards_compatibility.py:650: UserWarning: Removing legacy parameter method_kwargs from unit_locations extension
  warnings.warn(f"Removing legacy parameter {p} from {old_name} extension")
/opt/conda/lib/python3.9/site-packages/spikeinterface/core/waveforms_extracto

compute_waveforms:   0%|          | 0/6396 [00:00<?, ?it/s]

/opt/conda/lib/python3.9/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/opt/conda/lib/python3.9/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
/opt/conda/lib/python3.9/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/opt/conda/lib/python3.9/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
/opt/conda/lib/python3.9/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/opt/conda/lib/python3.9/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
/opt/conda/lib/python3.9/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in d

ValueError: zero-size array to reduction operation minimum which has no identity

In [2]:
ephys_opto_preprocessing('behavior_717121_2024-06-15_10-00-58', 'curated', 'soma')

behavior_717121_2024-06-15_10-00-58


/opt/conda/lib/python3.9/site-packages/spikeinterface/core/base.py:1129: UserWarning: Versions are not the same. This might lead to compatibility errors. Using spikeinterface==0.100.7 is recommended
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/spikeinterface/core/waveforms_extractor_backwards_compatibility.py:650: UserWarning: Removing legacy parameter return_scaled from spike_amplitudes extension
  warnings.warn(f"Removing legacy parameter {p} from {old_name} extension")
/opt/conda/lib/python3.9/site-packages/spikeinterface/core/waveforms_extractor_backwards_compatibility.py:650: UserWarning: Removing legacy parameter method_kwargs from unit_locations extension
  warnings.warn(f"Removing legacy parameter {p} from {old_name} extension")
/opt/conda/lib/python3.9/site-packages/spikeinterface/core/waveforms_extractor_backwards_compatibility.py:650: UserWarning: Removing legacy parameter sparsity from quality_metrics extension
  warnings.warn(f"Removing legacy parameter {p} from

In [4]:
opto_tagging_df = opto_plotting_session('behavior_717121_2024-06-15_10-00-58', 'curated', 'soma', plot=True, resp_thresh=0.4, lat_thresh=0.02)

/opt/conda/lib/python3.9/site-packages/spikeinterface/core/base.py:1129: UserWarning: Versions are not the same. This might lead to compatibility errors. Using spikeinterface==0.100.7 is recommended
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/spikeinterface/core/waveforms_extractor_backwards_compatibility.py:650: UserWarning: Removing legacy parameter return_scaled from spike_amplitudes extension
  warnings.warn(f"Removing legacy parameter {p} from {old_name} extension")
/opt/conda/lib/python3.9/site-packages/spikeinterface/core/waveforms_extractor_backwards_compatibility.py:650: UserWarning: Removing legacy parameter method_kwargs from unit_locations extension
  warnings.warn(f"Removing legacy parameter {p} from {old_name} extension")
/opt/conda/lib/python3.9/site-packages/spikeinterface/core/waveforms_extractor_backwards_compatibility.py:650: UserWarning: Removing legacy parameter sparsity from quality_metrics extension
  warnings.warn(f"Removing legacy parameter {p} from

In [3]:
ephys_opto_crosscorr('behavior_717121_2024-06-15_10-00-58', 'curated')
opto_wf_preprocessing('behavior_717121_2024-06-15_10-00-58', 'curated', 'soma', load_sorting_analyzer=False)

/opt/conda/lib/python3.9/site-packages/spikeinterface/core/base.py:1129: UserWarning: Versions are not the same. This might lead to compatibility errors. Using spikeinterface==0.100.7 is recommended
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/spikeinterface/core/waveforms_extractor_backwards_compatibility.py:650: UserWarning: Removing legacy parameter return_scaled from spike_amplitudes extension
  warnings.warn(f"Removing legacy parameter {p} from {old_name} extension")
/opt/conda/lib/python3.9/site-packages/spikeinterface/core/waveforms_extractor_backwards_compatibility.py:650: UserWarning: Removing legacy parameter method_kwargs from unit_locations extension
  warnings.warn(f"Removing legacy parameter {p} from {old_name} extension")
/opt/conda/lib/python3.9/site-packages/spikeinterface/core/waveforms_extractor_backwards_compatibility.py:650: UserWarning: Removing legacy parameter sparsity from quality_metrics extension
  warnings.warn(f"Removing legacy parameter {p} from

compute_waveforms:   0%|          | 0/6396 [00:00<?, ?it/s]

ValueError: Folder already exists /root/capsule/scratch/717121/behavior_717121_2024-06-15_10-00-58/ephys/opto/curated/opto_waveforms.zarr

In [6]:
pip install ipykernel

In [10]:
pip install Ipykernel

In [ ]:

Parallel(n_jobs=3)(delayed(process)(session) for session in session_list)

In [8]:
plt.close('all')
data_type = 'raw'
session = session_list[0]
opto_tagging_df = opto_plotting_session(session, data_type, 'soma', plot=True, resp_thresh=0.4, lat_thresh=0.02)
plt.close('all')
opto_tagged_spike_stability(session, data_type, 'soma', opto_tagging_df=opto_tagging_df)

/root/capsule/code/beh_ephys_analysis/opto_tagging.py:345: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  unit_qc = unit_qc.apply(pd.to_numeric, errors='ignore')


95 out of 271 units pass quality control
{'samples_to_keep': [-30, 60], 'y_neighbors_to_keep': 3, 'orginal_loc': False}


/root/capsule/code/beh_ephys_analysis/opto_tagging.py:386: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  opto_wf = opto_wf.apply(pd.to_numeric, errors='ignore')


Merging file 0 out of 271
Merging file 50 out of 271
Merging file 100 out of 271
Merging file 150 out of 271
Merging file 200 out of 271
Merging file 250 out of 271
PDF files in '/root/capsule/scratch/751004/behavior_751004_2024-12-19_11-50-37/ephys/opto/raw/figures' merged into '/root/capsule/scratch/751004/behavior_751004_2024-12-19_11-50-37/ephys/opto/raw/behavior_751004_2024-12-19_11-50-37_opto_tagging.pdf' successfully.
5 out of 271 units pass quality control and opto tagging


/root/capsule/code/beh_ephys_analysis/opto_tagging.py:458: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  opto_tagging_df_focus = opto_tagging_df_sorted[(opto_tagging_df_sorted['opto_pass'] == True) & (opto_tagging_df['decoder_label'] != 'artifact') & (opto_tagging_df['decoder_label'] != 'noise')].copy()


In [ ]:
session_list = ['']

In [ ]:
for session in session_list:
    ephys_opto_preprocessing(session, 'curated', 'soma')
    opto_wf_preprocessing(session, 'curated', 'soma', load_sorting_analyzer=False)
    opto_plotting_session(session, 'curated', plot=True)
    plt.close('all')

In [7]:
session = 'behavior_751004_2024-12-20_13-26-11'
data_type = 'raw'
target = 'soma'
target_unit_ids = [16]
# pass_df = opto_plotting_session(session, data_type, target, plot=True,  resp_thresh=0.5, lat_thresh=0.015, target_unit_ids=target_unit_ids)

In [ ]:
data_type = 'raw'
target = 'soma'
for session in session_list:
    combine_qc = opto_plotting_session(session, data_type, target, plot=True,  resp_thresh=0.5, lat_thresh=0.015)

In [8]:
session_dir = session_dirs(session)
opto_wf_csv = os.path.join(session_dir[f'opto_dir_{data_type}'], f'{session}_opto_waveform_metrics.csv')
opto_wf = pd.read_csv(opto_wf_csv)

In [1]:
pip3 install --no-cache-dir numpy

SyntaxError: invalid syntax (4263930493.py, line 1)